# Dicke quantum battery, coupling scheme: evaluate and export performance

Evaluates the performance of the trained data, and exports all data as readable files and pdf plots
#### Import modules

In [ ]:
import numpy as np
import sys
import os
sys.path.append(os.path.join('..','src'))
import extra
import sac_epi_envs

## Export trained data

This command allows to evaluate and export data for multiple training folders create either using ```0_train_coupling_case.ipynb``` or ```0_train_detuning_case.ipynb```.

In particular, let us assume that multiple training folders are in ```main_dir```. It loads all of them that satisfy the criteria in ```conditions_dict```. Here, we choose as criteria a fixed number of qubits $N$, i.e. ```nq``` in the code. In the manuscript, we did this for $N=12,14,16,18,20$, so we do the same in this code example. For each value  of ```nq```, that we denote with ```{nq_val}```, it will:
- create a folder named ```"nq={nq_val}"``` within the ```main_dir``` folder;
- create a file in ```main_dir``` called ```"performance_tau_nq={nq_val}.txt"```.

The folder ```"nq={nq_val}"``` contains the following files for every single training folder, which should correspond to a different value of tau (```{tau_val}``` is to be replaced with the value of tau of each training folder):
- ```"eval_tau={tau_val}.pdf"```: plot with the return ```G```, ```obj 0```, i.e. the energy, ```obj 1```, i.e. the ergotropy, and ```u```, the time-dependent control, as a function of the training step. Here the final deterministic policy is used to choose the actions. This plot is also shown in this Jupyter Notebook during the execution of the following block.
- ```"spectrum_weights_tau={tau_val}.pdf"```. Plot analyzing the performance of the final deterministic policy. First 3 panels correspond respectively to the projection of the quantum state onto the eigenstates of $\mathcal{\hat H}(t)$, the eigenstates of $\mathcal{\hat H}_{\rm C}$, and the eigenstates of $\mathcal{\hat H}_{\rm B}$; then the y represents the energy, and the x is time, and the color is the square of the projection of the state onto the eigenstates. The following panels show the entropy of all the TLS, the entropy of a single TLS, the single TLS ergotropy, and the time dependent control as a function of time.
-  ```"train_tau={tau_val}.pdf"```: pdf plot representing the training behaviour. This corresponds to the plot outputted during training (see ```0_train_coupling_case.ipynb```).  This plot is also shown in this Jupyter Notebook during the execution of the following block.
- ```"objs_extra_data_tau={tau_val}.txt"```: text file with the performance of the final deterministic policy. Every row is a time-step, and each of the 16 columns reprents respectively [0 = time step, 1 = time-dependent control, 2 = obj 0 (total battery energy), 3 = obj 1 (single TLS unit ergotropy), 4 = single TLS energy, 5 = single TLS entropy, 6 = single TLS energy variance, 7 = single TLS purity, 8 = battery energy, 9 = battery entropy, 10 = battery energy variance, 11 = cavity energy, 12 = cavity entropy, 13 = cavity energy variance, 14 = total energy (including coupling), 15 = total energy variance]
- ```"spectrum_weights_tau={tau_val}.npz"```: numpy compressed object with a ```"data"``` key. This is an array with shape (time_steps,hamil_index,spectrum,2). Time_steps indexes each timestep. hamil_index represents which hamiltonian we are projecting on, with 0=$\mathcal{\hat H}_{\rm C}+\mathcal{\hat H}_{\rm B}$, 1=$\mathcal{\hat H}_{\rm C}$, 2=$\mathcal{\hat H}_{\rm B}$. Spectrum is an integer for each eigenstate of the corresponding Hamiltonian. The final index is 0=(energy value of the eingenstate), 1=(square projection of the state on the corresponding eigenstate).

The file ```"performance_tau_nq={nq_val}.txt"``` contains the performance of all the files with the given $N$. In particular, each row is a different value of tau (i.e. each row is a separate training), and each column represents the same exact quantities in ```"objs_extra_data_tau={tau_val}.txt"```, but evaluated at the final time, and the value of the time-dependent control (index=1), is replaced with the return.



In [ ]:
#folder where the training folders are
main_dir = "../data/"  
#which quantity is in the first column of the "performance_tau_nq={nq_val}.txt" file. Not recommended changing this
x_quantity = "tau"  
#the quantities here present will be appended to the file names to distinguish them. Not recommended changing this
quantities_for_indexing = ["tau"] 
   
#loop over values of nq present in the main_dir
for nq in range(16,18,2):
    #only load training folders with this criteria. This way we get a separate txt file with the performance for 
    #each value of nq
    conditions_dict = {"nq": f"{nq}"} 
    extra.export_performance_and_protocol_files(main_dir, x_quantity, conditions_dict=conditions_dict,
                                        quantities_for_indexing=quantities_for_indexing, show_single_plot=False,
                                        exp_folder_name = f"nq={nq}",multiply_nc=2)

## Choose best run out of multiple identical ones

Let us assume we ran the same optimizations two times, i.e. we perform two repetitions. For each one, we exported the performance using the previous function into two distinc folders ```repetition_1``` and ```repetition_2```. The following creates a new folder ```destination_folder``` with the same exported data, but only selecting the best performance between the two folders. 

It further creates a file called ergo_mean_std_nq={nq_val}.txt for each value of nq that has different values of tau by row, and the 3 columns are the value of tau, the average ergotropy across mutiple runs, and the standard deviation of the ergotropy across multiple runs.

For this to work, the two repetitions should contain identical runs. This function can compare an arbitrary number of repetitions (in the Manuscript, we display the best of 5 repetitions).

In [ ]:
#list of folders, one for each repetition
compare_folders =  [r"../important_data/repetition_1", r"../important_data/repetition_2"]    
#destination folder where the only the best of two repetitions is reported
destination_folder = "../important_data/best_of_2_repetitions"
#list of values of nq to be compared
nq_vals = np.arange(12,22,2)

#run the command
extra.create_best_of_multiple_runs(compare_folders, destination_folder, nq_vals)

## Create on-off protocol reference data

The following can be used to export the same data, but assuming we use the on-off protcol instead of the RL policies

In [ ]:
#nq_vals for which we want to export the on-off protocol performance
nq_vals = np.arange(12,22,2)
#folder where the data is exported
exp_folder = "../important_data/on_off_data"

for nq in nq_vals:
    #create the environment parameters
    dt=0.2
    multiply_nc=2
    tau = 5.6
    env_params = {
            "wc": 1.,                       
            "nc": nq*2,
            "nc_i": nq,
            "wq": 1.,
            "nq": nq,   
            "min_u": -0.3,                      
            "max_u": +0.3,  
            "dt": dt,
            "tau": tau,     
            "reward_coeff": 1.,
            "quantity": "ergotropy"
        } 
    #create an action corresponding the the on-off protocol
    action = np.array([0.3], dtype=np.float32)
    #extra string to append to the file
    extra_str = f"_nq={nq}"
    #evaluate the performance of the on-off
    extra.test_policy(sac_epi_envs.DickeBatteryOneControlCoupling, env_params,
                         lambda o: action, False,suppress_show=False,
                         actions_to_plot=500,dont_clear_output=True,
                         actions_ylim=None,
                         export_pdf_location=os.path.join(exp_folder, f"eval{extra_str}.pdf"),
                         export_objs_and_entanglement_location=os.path.join(exp_folder, f"objs_extra_data{extra_str}.txt"),
                         export_spectrum_weights_location = os.path.join(exp_folder, f"spectrum_weights{extra_str}.npz"),
                         export_spetrum_weights_pdf_location = os.path.join(exp_folder, f"spectrum_weights{extra_str}.pdf"),dt=dt,
                         multiply_nc=multiply_nc)